In [1]:
from datasets import load_dataset,DatasetDict
from sklearn.model_selection import train_test_split
import io
import os
from PIL import Image

/home/omar/TUM/05_projects/Image_generation_with_VAE/vae_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
ds = load_dataset("Karthik11232/human_face_generation")
#split the dataset 
train_test_set = ds['train'].train_test_split(test_size=0.2)
train_val_set = train_test_set['train'].train_test_split(test_size=0.2)

ds = DatasetDict({
    'train': train_test_set['train'],
    'test': train_test_set['test'],
    'validation': train_val_set['test']
})
print (ds)
print(ds.shape)
print(train_test_set.shape)
print(train_val_set.shape)

DatasetDict({
    train: Dataset({
        features: ['image'],
        num_rows: 2132
    })
    test: Dataset({
        features: ['image'],
        num_rows: 533
    })
    validation: Dataset({
        features: ['image'],
        num_rows: 427
    })
})
{'train': (2132, 1), 'test': (533, 1), 'validation': (427, 1)}
{'train': (2132, 1), 'test': (533, 1)}
{'train': (1705, 1), 'test': (427, 1)}


In [5]:
# Save the datasets to disk
output_dir = './human_face_generation_split'
os.makedirs(output_dir, exist_ok=True)

# Function to save images
def save_images(dataset, split_name):
    split_dir = os.path.join(output_dir, split_name)
    os.makedirs(split_dir, exist_ok=True)
    
    for idx, sample in enumerate(dataset):
        #print(f'{idx} {sample["image"]}')
        image=sample["image"]
        #buf = io.BytesIO()
        #image = Image.open(sample['image'])
        #print(image)
        #image.save(buf, 'jpeg')
        image_path = os.path.join(split_dir, f'image_{idx}.png')
        #print(image_path)

        image.save(image_path,"jpeg")

# Save each split
save_images(ds['train'], 'train')
save_images(ds['validation'], 'validation')
save_images(ds['test'], 'test')